In [2]:
from AHI_AC import *

# Set spatial resolution
# 0.01deg ==> 0.01, 0.02deg ==> 0.02
res = 0.02

# Set latitude upper and bottom bounds
lat_up, lat_bottom = 60, -60

# Set longitude left and right bounds
lon_left, lon_right = 85, 205

# Set save folder
target = '/data01/GEO/SRDATA/'

# See main.py
row,col,row_AHI,col_AHI,AHI_lat,AHI_lon,AHI_VZA, AHI_VAA,AHI_AL,Water_idx,Landmask = main_read(res,lat_up, lat_bottom,lon_left, lon_right)

# Read LUT and interpolate with LUT
# Processing band can be set by yourself!

interpolation = LUT_interpolator(5)
fn1_5, fn2_5, fn3_5 = interpolation._interpolate()
interpolation = LUT_interpolator(6)
fn1_6, fn2_6, fn3_6 = interpolation._interpolate()

# Set necessary parameters
band_type = {'5': 'sir', '6': 'sir', }
band_num = {'5': '01', '6': '02'}
calculate_6s = {'5': calculate_6s_band5, '6': calculate_6s_band6}
fn1 = {'5': fn1_5, '6': fn1_6}
fn2 = {'5': fn2_5, '6': fn2_6}
fn3 = {'5': fn3_5, '6': fn3_6}


In [3]:
def SR_Process(band):
    start_time = T.time()

    # Download AHI
    print(f"Start download AHI for Band{band}")
    S_Read_AHI_time = T.time()
    num_threads = 4  # 


    H8 = H8_data('liwei', 'liwei00', band_type[band], band_num[band], date, folder_original)
    NAS_file_path,Target_path = H8.file_path(),H8.target_path()
    multi_threaded_copy(NAS_file_path, Target_path, num_threads)
    decompression(Target_path)    
    AHI_data = H8.read_H8data(Target_path[:-4])

    if AHI_data == 'No data':
        print(f"No AHI data for Band{band}, pass")
        return None
    AHI_data = AHI_data / np.cos(np.radians(AHI_SZA))
    E_Read_AHI_time = T.time()
    print(f"Finish download AHI for Band{band}")
    print('AHI ORG data processing cost : {}s'.format(round((E_Read_AHI_time-S_Read_AHI_time),4)))
    
    
    # SR calculate
    print(f'Start Calculate SR for Band{band}')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=njob)(delayed(calculate_6s[band])(fn1[band], fn2[band], fn3[band], Landmask, Aerosol_type, *params[band], AHI_data, i) for i in range(row_AHI))
    SR = np.full((row_AHI,col_AHI), np.nan)
    for i in range(row_AHI):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print(f'Calculate SR for Band{band} finished')
    print('Calculate SR cost : {}s'.format(round((E_SR_time-S_SR_time),4)))

    # Save file and remove download input data
    print(f'Start Save SR file for Band{band}')
    S_SAVE_time = T.time()
    SR = np.array(SR).reshape(row_AHI,col_AHI)
    
    SR = np.around(SR, 4) * 10000
    
    SR = np.clip(SR, -32767, 32767)
    SR = np.nan_to_num(SR, nan=-32768)
    SR = np.where(np.isnan(SR), -32768, SR)

    SR_file = open(folder_AC + f'H08_{YYYY}{MM}{DD}_{HH}{MIN}_B0{band}_SR.dat', 'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    E_SAVE_time = T.time()
    print(f'Save Band{band} SR finished')
    print('Save file cost : {}s'.format(round((E_SAVE_time-S_SAVE_time),4)))
    # if band == '4':
    #     remove_original_file(folder_original)
    #     print("delete file finish")
    
    end_time = T.time()
    TIME = end_time - start_time
    print('Band{} cost time: {:} secs, {:.1f} mins\n'.format(band, TIME, TIME / 60))

In [ ]:
error_list = []

njob = 39
date_start = '2020-01-08 13:00'
date_end = '2020-01-31 23:50'
date_t = dt.timedelta(minutes=10)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
# print("Start {} ".format(date_start[:-6]))
YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
while date_time_now <= date_e:
    try:
        date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
        YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)   
        hour = int(date[8:10])
        print(date_time_now)
        if  hour >= 14 and hour <= 22:
        # if  hour >= 8:
            print(f"{date_time_now} not in time range, pass")
        else:
            S_time = T.time()
            print("Start processing {}".format(date))

            # make dir
            folder_original = f"{target}{date}_original/"
            TOA_PATH = f'/data01/GEO/SRDATA_LAI/{date}_original/'
            folder_AC = f"{target}{YYYY}/{MM}/{DD}/"
            mkdir(folder_original)
            mkdir(folder_AC)

            # Solar angle
            print('Start read solar angle')
            S_ANGLE_time = T.time()
            AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
            AHI_SZA,AHI_SAA = AHI_SZA[row:row+row_AHI,col:col+col_AHI],AHI_SAA[row:row+row_AHI,col:col+col_AHI]
            RAA = abs(AHI_SAA - AHI_VAA)
            RAA[RAA>180]=360-RAA[RAA>180]
            print('Finish read solar angle')
            E_ANGLE_time = T.time()
            print('Read angle cost : {}s'.format(round((E_ANGLE_time-S_ANGLE_time),4)))

            # Atmosphereic data
            print('Start read CAMS')
            S_ATMOS_time = T.time()
            CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
            Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
            print('Finish read CAMS')
            E_ATMOS_time = T.time()
            print('Read Atmospheric data cost : {}s'.format(round((E_ATMOS_time-S_ATMOS_time),4)))

            params = {
                        '5': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                        '6': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
                    }

            for band in ['5', '6']:
                SR_Process(band)
            E_time = T.time()
            print('Total time cost : {}s'.format(round((E_time-S_time),4)))
        # else:
            # print(f"{date_time_now} not in time range, pass")
    except:
        error_list.append(date)
    date_time_now = date_time_now + date_t

# WITH INPUT DATE

In [3]:
date_list = [
 '201808060320',
 '201808090210',
 '201808130320',
 '201808180200',
 '201808290320',
 '201808300220',
 '201808300230',
 '201808310310',
 '201808060610',
 '201808090510',
 '201808100550',
 '201808110450',
 '201808140520',
 '201808150610',
 '201808170550',
 '201808180500',
 '201808240600'
]

In [ ]:
njob = 20

for DATE in date_list:
    
    YYYY,MM,DD,HH,MIN,date= DATE[0:4],DATE[4:6],DATE[6:8],DATE[8:10],DATE[10:12],DATE
    print(DATE)
    # if  hour >=9 and hour <= 22:

    S_time = T.time()
    print("Start processing {}".format(date))

    # make dir
    folder_original = f"{target}{date}_original/"
    TOA_PATH = f'/data01/GEO/SRDATA_LAI/{date}_original/'
    folder_AC = f"{target}{YYYY}/{MM}/{DD}/"
    mkdir(folder_original)
    mkdir(folder_AC)

    # Solar angle
    print('Start read solar angle')
    S_ANGLE_time = T.time()
    AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
    AHI_SZA,AHI_SAA = AHI_SZA[row:row+row_AHI,col:col+col_AHI],AHI_SAA[row:row+row_AHI,col:col+col_AHI]
    RAA = abs(AHI_SAA - AHI_VAA)
    RAA[RAA>180]=360-RAA[RAA>180]
    print('Finish read solar angle')
    E_ANGLE_time = T.time()
    print('Read angle cost : {}s'.format(round((E_ANGLE_time-S_ANGLE_time),4)))

    # Atmosphereic data
    print('Start read CAMS')
    S_ATMOS_time = T.time()
    CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
    Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
    print('Finish read CAMS')
    E_ATMOS_time = T.time()
    print('Read Atmospheric data cost : {}s'.format(round((E_ATMOS_time-S_ATMOS_time),4)))

    params = {
                '5': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                '6': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
            }

    for band in ['5', '6']:
        SR_Process(band)
    E_time = T.time()
    print('Total time cost : {}s'.format(round((E_time-S_time),4)))
    # else:
        # print(f"{date_time_now} not in time range, pass")
